In [36]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

N = 50 # number of samples
L = 300 # length of each sample (number of values for each sine wave)
T = 20 # width of the wave

x = np.empty((N,L), np.float32) # instantiate empty array
random_shift = np.random.randint(-4*T, 4*T, N).reshape(N,1)
x[:] = np.arange(L) + random_shift
y = np.sin(x/1.0/T).astype(np.float32)


# every row of x is a list of positions x in the sin function, where each row is shifted by an amount
# y is the output of the sin function, sin(x) at every point x


class LSTM(nn.Module):
    def __init__(self, hidden_layers=64):
        super(LSTM, self).__init__()
        self.hidden_layers = hidden_layers
        # lstm1, lstm2, linear are all layers in the network
        self.lstm1 = nn.LSTMCell(1, self.hidden_layers)
        self.lstm2 = nn.LSTMCell(self.hidden_layers, self.hidden_layers)
        self.linear = nn.Linear(self.hidden_layers, 1)
        
    def forward(self, y, future_preds=0):
        outputs, num_samples = [], y.size(0)
        h_t = torch.zeros(num_samples, self.hidden_layers, dtype=torch.float32)
        c_t = torch.zeros(num_samples, self.hidden_layers, dtype=torch.float32)
        h_t2 = torch.zeros(num_samples, self.hidden_layers, dtype=torch.float32)
        c_t2 = torch.zeros(num_samples, self.hidden_layers, dtype=torch.float32)
        
        for time_step in y.split(1, dim=1):
            # N, 1
            h_t, c_t = self.lstm1(time_step, (h_t, c_t)) # initial hidden and cell states
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) # new hidden and cell states
            output = self.linear(h_t2) # output from the last FC layer
            outputs.append(output)
            
        for i in range(future_preds):
            # this only generates future predictions if we pass in future_preds>0
            # mirrors the code above, using last output/prediction as input
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
            
        # transform list to tensor    
        outputs = torch.cat(outputs, dim=1)
        return outputs
    
    
train_input = torch.from_numpy(y[3:, :-1]) # (97, 999)
train_target = torch.from_numpy(y[3:, 1:]) # (97, 999)

test_input = torch.from_numpy(y[:3, :-1]) # (3, 999)
test_target = torch.from_numpy(y[:3, 1:]) # (3, 999)

model = LSTM()
criterion = nn.MSELoss()
optimiser = optim.LBFGS(model.parameters(), lr=0.08)

def training_loop(n_epochs, model, optimiser, loss_fn, 
                  train_input, train_target, test_input, test_target):
    for i in range(n_epochs):
        print("epoch", i)
        def closure():
            optimiser.zero_grad()
            out = model(train_input)
            loss = loss_fn(out, train_target)
            loss.backward()
            return loss
        optimiser.step(closure)
        with torch.no_grad():
            future = L
            pred = model(test_input, future_preds=future)
            # use all pred samples, but only go to 999
            loss = loss_fn(pred[:, :-future], test_target)
            y = pred.detach().numpy()
        # draw figures
        plt.figure(figsize=(12,6))
        plt.title(f"Step {i+1}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        n = train_input.shape[1] # 999
        def draw(yi, colour):
            plt.plot(np.arange(n), yi[:n], colour, linewidth=2.0)
            plt.plot(np.arange(n, n+future), yi[n:], colour+":", linewidth=2.0)
        draw(y[0], 'r')
        draw(y[1], 'b')
        draw(y[2], 'g')
        plt.savefig("predict%d.png"%i, dpi=200)
        plt.close()
        # print the loss
        out = model(train_input)
        loss_print = loss_fn(out, train_target)
        print("Step: {}, Loss: {}".format(i, loss_print))

training_loop(40, model, optimiser, criterion, train_input, train_target, test_input, test_target)
        


epoch 0
Step: 0, Loss: 0.03965402767062187
epoch 1
Step: 1, Loss: 0.02076491340994835
epoch 2
Step: 2, Loss: 0.014110084623098373
epoch 3
Step: 3, Loss: 0.002376302843913436
epoch 4
Step: 4, Loss: 0.0019026579102501273
epoch 5
Step: 5, Loss: 0.000939610879868269
epoch 6
Step: 6, Loss: 0.0007219105609692633
epoch 7
Step: 7, Loss: 0.0003942791372537613
epoch 8
Step: 8, Loss: 0.00032781570916995406
epoch 9
Step: 9, Loss: 0.00022571887529920787
epoch 10
Step: 10, Loss: 0.000144373785587959
epoch 11
Step: 11, Loss: 7.323654426727444e-05
epoch 12
Step: 12, Loss: 4.331710806582123e-05
epoch 13
Step: 13, Loss: 2.7907379262614995e-05
epoch 14
Step: 14, Loss: 2.0822017177124508e-05
epoch 15
Step: 15, Loss: 1.945869553310331e-05
epoch 16
Step: 16, Loss: 1.8340404494665563e-05
epoch 17
Step: 17, Loss: 1.5309387890738435e-05
epoch 18
Step: 18, Loss: 1.4679071682621725e-05
epoch 19
Step: 19, Loss: 1.4097829989623278e-05
epoch 20
Step: 20, Loss: 1.2855682143708691e-05
epoch 21
Step: 21, Loss: 1.22076